In [1]:
import pandas as pd

# 2WikiMultiHopQA

In [2]:
df_wiki_train = pd.read_json('data/QA-dataset/data/2wikimultihopQA/train.json')
df_wiki_dev = pd.read_json('data/QA-dataset/data/2wikimultihopQA/dev.json')
df_wiki_test = pd.read_json('data/QA-dataset/data/2wikimultihopQA/test.json')

In [3]:
df_wiki_corpus = pd.concat([df_wiki_train, df_wiki_dev, df_wiki_test])
df_wiki_corpus.drop(columns=['type', 'supporting_facts', 'evidences'], inplace=True)
df_wiki_corpus

,_id,question,context,answer
0,13f5ad2c088c11ebbd6fac1f6bf848b6,Are director of film Move (1970 Film) and dire...,"[[Stuart Rosenberg, [Stuart Rosenberg (August ...",no
1,3057c6c4086111ebbd5dac1f6bf848b6,Do both films The Falcon (Film) and Valentin T...,"[[The Falcon Takes Over, [The Falcon Takes Ove...",no
2,89bc944808a111ebbd79ac1f6bf848b6,"Which film whose director is younger, Charge I...","[[Danger: Diabolik, [Danger:, Diabolik is a 1...",Danger: Diabolik
3,633f80660bdd11eba7f7acde48001122,What is the date of birth of Mina Gerhardsen's...,"[[Pamela Jain, [Pamela Jain is an Indian playb...",13 June 1946
4,2dc3f9740bda11eba7f7acde48001122,What nationality is the director of film Weddi...,"[[Weekend in Paradise (1931 film), [Weekend in...",Hungarian
...,...,...,...,...
12571,def0391c0bdd11eba7f7acde48001122,Where was the place of burial of Beatrice Of B...,"[[Margaret of Bohemia, Duchess of Bavaria, [Ma...",
12572,8792d5e40bdd11eba7f7acde48001122,Where was the wife of Puven Pather born?,"[[Papianilla (wife of Tonantius Ferreolus), [P...",
12573,d20deaa8085b11ebbd5cac1f6bf848b6,Do director of film Prisoner Of The Night (Fil...,"[[Hanro Smitsman, [Hanro Smitsman, born in 196...",
12574,153fbd2f094e11ebbdaeac1f6bf848b6,Are the movies Gladiatress and Never Trouble T...,"[[Renee Clama, [Renee Clama( 1908 – 1990) was ...",


In [4]:
df_wiki_corpus = df_wiki_corpus.explode('context', ignore_index=True)
df_wiki_corpus['context'] = df_wiki_corpus['context'].apply(lambda row: row[0] + ': ' + ' '.join(row[1]))
df_wiki_corpus

,_id,question,context,answer
0,13f5ad2c088c11ebbd6fac1f6bf848b6,Are director of film Move (1970 Film) and dire...,"Stuart Rosenberg: Stuart Rosenberg (August 11,...",no
1,13f5ad2c088c11ebbd6fac1f6bf848b6,Are director of film Move (1970 Film) and dire...,Méditerranée (1963 film): Méditerranée is a 19...,no
2,13f5ad2c088c11ebbd6fac1f6bf848b6,Are director of film Move (1970 Film) and dire...,Move (1970 film): Move is a 1970 American come...,no
3,13f5ad2c088c11ebbd6fac1f6bf848b6,Are director of film Move (1970 Film) and dire...,Ian Barry (director): Ian Barry is an Australi...,no
4,13f5ad2c088c11ebbd6fac1f6bf848b6,Are director of film Move (1970 Film) and dire...,Peter Levin: Peter Levin is an American direct...,no
...,...,...,...,...
1926055,8559542c086a11ebbd5fac1f6bf848b6,Who was born first out of William John Hancock...,John L. Simpson: John L. Simpson( born 2 Septe...,
1926056,8559542c086a11ebbd5fac1f6bf848b6,Who was born first out of William John Hancock...,Oggy Boytchev: Ognian Boytchev( Bulgarian: Огн...,
1926057,8559542c086a11ebbd5fac1f6bf848b6,Who was born first out of William John Hancock...,Henry Moore (cricketer): Henry Walter Moore( 1...,
1926058,8559542c086a11ebbd5fac1f6bf848b6,Who was born first out of William John Hancock...,Wale Adebanwi: Wale Adebanwi( born 1969) is a ...,


In [5]:
df_wiki_corpus.to_csv('data/wikihop_corpus.csv')

# MuSiQue

In [6]:
df_musique_train = pd.read_json('data/QA-dataset/data/musique/musique_full_v1.0_train.jsonl', lines=True)
df_musique_dev = pd.read_json('data/QA-dataset/data/musique/musique_full_v1.0_dev.jsonl', lines=True)
df_musique_test = pd.read_json('data/QA-dataset/data/musique/musique_full_v1.0_test.jsonl', lines=True)

In [7]:
df_musique_corpus = pd.concat([df_musique_train, df_musique_dev, df_musique_test])
df_musique_corpus.drop(columns=['question_decomposition', 'answer_aliases', 'answerable'], inplace=True)
df_musique_corpus

,id,paragraphs,question,answer
0,2hop__42543_20093,"[{'idx': 0, 'title': 'All Things in Time', 'pa...",What year did the writer of Crazy Little Thing...,1991
1,2hop__269805_135710,"[{'idx': 0, 'title': 'Milton F. Pavlic', 'para...",What is the country where Nissedal is located ...,north
2,2hop__568389_161223,"[{'idx': 0, 'title': 'Bugabula', 'paragraph_te...",What is the highest point in the country where...,"1,400 metres"
3,2hop__131693_45439,"[{'idx': 0, 'title': 'United States Declaratio...",Who from the state with the Routzahn-Miller Fa...,Charles Carroll
4,2hop__150985_444333,"[{'idx': 0, 'title': 'Journal of Geographical ...",Who founded the publisher of Journal of Media ...,George Routledge
...,...,...,...,...
4913,3hop1__157807_19788_19761,"[{'idx': 0, 'title': 'Red-bellied black snake'...",Which missionary helped spread the religion wi...,NaN
4914,3hop1__26222_36635_87083,"[{'idx': 0, 'title': 'Chrysler Pacifica (1999 ...",Who hold veto power in the organization in whi...,NaN
4915,2hop__129784_40413,"[{'idx': 0, 'title': 'Tenochtitlan', 'paragrap...",What percentage of Australian pears are grown ...,NaN
4916,2hop__104350_8583,"[{'idx': 0, 'title': 'Slavs', 'paragraph_text'...",Who did the Slavs have contact with in the cou...,NaN


In [8]:
df_musique_corpus = df_musique_corpus.explode('paragraphs', ignore_index=True)
df_musique_corpus['paragraphs'] = df_musique_corpus['paragraphs'].apply(lambda row: row['title'] + ': ' +  row['paragraph_text'])
df_musique_corpus

,id,paragraphs,question,answer
0,2hop__42543_20093,All Things in Time: All Things in Time is an a...,What year did the writer of Crazy Little Thing...,1991
1,2hop__42543_20093,Goin' Out of My Head: ``Goin 'Out of My Head '...,What year did the writer of Crazy Little Thing...,1991
2,2hop__42543_20093,Every Little Thing U Do: ``Every Little Thing ...,What year did the writer of Crazy Little Thing...,1991
3,2hop__42543_20093,She Don't Love You: ``She Do n't Love You ''is...,What year did the writer of Crazy Little Thing...,1991
4,2hop__42543_20093,Crazy Desire: Crazy Desire (originally titled ...,What year did the writer of Crazy Little Thing...,1991
...,...,...,...,...
992344,3hop2__131820_39793_61019,Citibank Australia: Citibank Australia was ope...,What was the capital of the state where WEKL i...,NaN
992345,3hop2__131820_39793_61019,Crocker National Bank: Crocker National Bank w...,What was the capital of the state where WEKL i...,NaN
992346,3hop2__131820_39793_61019,"ICICI Bank: ICICI Bank, stands for Industrial ...",What was the capital of the state where WEKL i...,NaN
992347,3hop2__131820_39793_61019,"Conondale, Queensland: Conondale is a town in ...",What was the capital of the state where WEKL i...,NaN


In [9]:
df_musique_corpus.to_csv('data/musique_corpus.csv')